In [1]:
# import tcia2nifti as t2n
import glob
import os
from tqdm import tqdm

import pandas as pd
import pathlib as plb
import tempfile
import dicom2nifti
import SimpleITK as sitk
import shutil
import nibabel as nib

import numpy as np
import pandas as pd

In [2]:
data_paths = glob.glob('d:/AutoPET_data/CT_dir/*/*/*/*/CT.nii.gz')
valid_candidates = [] 
ori_dim = []
ori_shape = [] 
new_shape = []
cnt = 0 
new_spacing = (2.03642,  2.03642, 3.)
for path in tqdm(data_paths):
    volume = sitk.ReadImage(path)
    ret = sitk.GetArrayFromImage(volume)
    print(type(ret))
    break
    original_spacing = volume.GetSpacing()
    original_size    = volume.GetSize()
    new_size = [int(round(osz*ospc/nspc)) for osz,ospc,nspc in zip(original_size, original_spacing, new_spacing)]
    corresponding_size = sitk.Resample(volume, new_size, sitk.Transform(), sitk.sitkLinear,
                         volume.GetOrigin(), new_spacing, volume.GetDirection(), 0,
                         volume.GetPixelID())
    # print(f'{original_spacing}, {original_size}-->{corresponding_size.GetSize()}')
    
    H, W, D = corresponding_size.GetSize() 
    if H < 128:
        continue
    
    if W < 128:
        continue
    
    if D < 96:
        continue
    
    valid_candidates.append(path)
    ori_dim.append(str(original_spacing))
    ori_shape.append(str(original_size))
    new_shape.append(str(corresponding_size.GetSize()))
    

  0%|          | 0/8202 [00:02<?, ?it/s]

<class 'SimpleITK.SimpleITK.Image'>


In [15]:
df = pd.DataFrame(np.stack([ori_dim, ori_shape, new_shape, valid_candidates]).T, columns=['ori_dim','ori_shape','new_shape','path'])
df.to_csv('D:/git/autoPET/val_CT_list(128x128x96).csv')


In [16]:
data_paths1 = glob.glob('d:\\AutoPET_data\\MSD dataset/*/*/imagesTr/*')
data_paths2 = glob.glob('d:\\AutoPET_data\\MSD dataset/*/*/imagesTs/*')
data_paths = data_paths1 + data_paths2
valid_candidates = [] 
ori_dim = []
ori_shape = [] 
new_shape = []
cnt = 0 
new_spacing = (2.03642,  2.03642, 3.)
for path in data_paths:
    volume = sitk.ReadImage(path)
    original_spacing = volume.GetSpacing()
    original_size    = volume.GetSize()
    new_size = [int(round(osz*ospc/nspc)) for osz,ospc,nspc in zip(original_size, original_spacing, new_spacing)]
    corresponding_size = sitk.Resample(volume, new_size, sitk.Transform(), sitk.sitkLinear,
                         volume.GetOrigin(), new_spacing, volume.GetDirection(), 0,
                         volume.GetPixelID())
    
    H, W, D = corresponding_size.GetSize() 
    if H < 128:
        continue
    
    if W < 128:
        continue
    
    if D < 96:
        continue
    
    print(f'{str(original_spacing):45s}, {original_size}-->{corresponding_size.GetSize()}')
    
    valid_candidates.append(path)
    ori_dim.append(str(original_spacing))
    ori_shape.append(str(original_size))
    new_shape.append(str(corresponding_size.GetSize()))
    

(0.9707030057907104, 0.9707030057907104, 2.5), (512, 512, 121)-->(244, 244, 101)
(0.9511719942092896, 0.9511719942092896, 5.0), (512, 512, 91)-->(239, 239, 152)
(0.8144530057907104, 0.8144530057907104, 5.0), (512, 512, 96)-->(205, 205, 160)
(0.7890620231628418, 0.7890620231628418, 5.0), (512, 512, 97)-->(198, 198, 162)
(0.78125, 0.78125, 5.0)                      , (512, 512, 87)-->(196, 196, 145)
(0.6445310115814209, 0.6445310115814209, 5.0), (512, 512, 77)-->(162, 162, 128)
(0.8046879768371582, 0.8046879768371582, 5.0), (512, 512, 92)-->(202, 202, 153)
(0.9316409826278687, 0.9316409826278687, 3.0), (512, 512, 97)-->(234, 234, 97)
(0.9277340173721313, 0.9277340173721313, 2.5), (512, 512, 147)-->(233, 233, 122)
(0.7539060115814209, 0.7539060115814209, 5.0), (512, 512, 98)-->(190, 190, 163)
(0.7695310115814209, 0.7695310115814209, 5.0), (512, 512, 98)-->(193, 193, 163)
(0.6601560115814209, 0.6601560115814209, 3.0), (512, 512, 134)-->(166, 166, 134)
(0.6835939884185791, 0.683593988418579

In [17]:
df = pd.DataFrame(np.stack([ori_dim, ori_shape, new_shape, valid_candidates]).T, columns=['ori_dim','ori_shape','new_shape','path'])
df.to_csv('D:/git/autoPET/val_CT_list_MSD(128x128x96).csv')


In [19]:
len(valid_candidates)

290

In [ ]:
df = pd.read_csv('D:/git/autoPET/val_PET_list(Not 9999exist).csv')

In [ ]:
target_root_dir = 'D:\\AutoPET_data\\Valid_PET(SUV)'

for path in tqdm(df['path']):
    chunk = path.split('/') 
    
    relative = chunk[2]
    relative = relative.split('\\')
    
    
    target = '\\'.join(relative[1:-2]) # 이게 어떤 파일인지 굳이 안 필요함 
    filename = relative[-1]
    copy = os.path.join(target_root_dir, target)
    if not os.path.isdir(copy):
        os.makedirs(copy)
        
    shutil.copy(path, os.path.join(copy,filename))


In [ ]:
autopet = nib.load('F:\\0_AutoPET\\autoPET_niiz\\PETCT_0af7ffe12a\\08-12-2005-NA-PET-CT Ganzkoerper  primaer mit KM-96698\\SUV.nii.gz')
print(autopet.header)

In [ ]:
f_paths = ["F:/0_AutoPET/FDG-PET-CT-Lesions", "F:/1_Lung-PET-CT-Dx/Lung-PET-CT-Dx", "F:/2_Head-Neck-PET-CT/Head-Neck-PET-CT", "F:/3_ACRIN-NSCLC-FDG-PET/ACRIN-NSCLC-FDG-PET", "F:/4_RIDER Lung PET-CT/RIDER Lung PET-CT", "F:/11_CPTAC-CM/CPTAC-CCRCC", "F:/11_CPTAC-CM/CPTAC-CM", "F:/14_NSCLC Radiogenomics/NSCLC Radiogenomics", "F:/15 LIDC-IDRI (LUNA 16)/LIDC-IDRI", "F:/16  COVID-19-NY-SBU/COVID-19-NY-SBU", "F:/19 MIDRC-RICORD-1B (COVID-19)/MIDRC-RICORD-1B"]
e_paths = ["D:/AutoPET_data/4 HNSCC/HNSCC", "D:/AutoPET_data/7 ACRIN-FLT-Breast/ACRIN-FLT-Breast", "D:/AutoPET_data/8 CT COLONOGRAPHY/CT COLONOGRAPHY", "D:/AutoPET_data/12 Anti-PD-1_MELANOMA/Anti-PD-1_MELANOMA", "D:/AutoPET_data/13 Anti-PD-1_Lung/Anti-PD-1_Lung"]
paths = f_paths + e_paths


for path in tqdm(df['path']):
    chunk = path.split('/') 
    
    relative = chunk[2]
    relative = relative.split('\\')
    
    
    target = '\\'.join(relative[1:-2]) # 이게 어떤 파일인지 굳이 안 필요함 
    
    